# Train software matching algorithm

This is the follow-on from the previous notebook "Train vendor matching algorithm".

The training proceeds in a similar manner:

* First the algorithm is tuned for typical data by using a grid search.
* Next the ML classifier is run on the training data using the optimum parameters.
* Finally the trained model is stored for future use.

## Read in the software training data

Read in the manually labelled software training data.

Format it and convert to two numpy arrays for input to the scikit-learn ML algorithm.

In [12]:
# Initialize

import pandas as pd
import numpy as np

try:
    df_label_software = pd.io.parsers.read_csv(
                            "/home/jovyan/work/vulnmine/vulnmine_data/label_software.csv",
                            error_bad_lines=False,
                            warn_bad_lines=True,
                            quotechar='"',
                            encoding='utf-8')
except IOError as e:
    print('\n\n***I/O error({0}): {1}\n\n'.format(
                e.errno, e.strerror))

# except ValueError:
#    self.logger.critical('Could not convert data to an integer.')
except:
    print(
        '\n\n***Unexpected error: {0}\n\n'.format(
            sys.exc_info()[0]))
    raise

# Number of records / columns

df_label_software.shape

(22262, 18)

In [13]:
# Print out some sample values

df_label_software.sample(5)

,Unnamed: 0,vendor_X,software_X,title_X,DisplayName0,release_X,Version0,fz_ratio,fz_ptl_ratio,fz_tok_set_ratio,fz_ptl_tok_sort_ratio,fz_uwratio,fz_rel_ratio,fz_rel_ptl_ratio,t_cve_name,titlX_len,DsplyNm0_len,match
4608,4920,adobe,acrobat,Adobe Acrobat X (10.0.3),Adobe Acrobat 9.5.5 - CPSID_83708,10.0.3,-,53,60,100,54,85,0,0,cpe:/a:adobe:acrobat:10.0.3,24,33,0
12044,12590,adobe,flash_player,Adobe Flash Player 10.1.95.2,Neolane v6.0 6.1.0 Build 8152,10.1.95.2,-,38,33,100,43,46,0,0,cpe:/a:adobe:flash_player:10.1.95.2,28,29,0
11142,11664,adobe,flash_player,Adobe Flash MX 2004 7.0.60.0,Neolane v6.0 6.1.0 Build 8113,7.0.60.0,-,30,33,100,43,40,0,0,cpe:/a:adobe:flash_player:7.0.60.0,28,29,0
9204,9678,adobe,flash_player,Adobe Flash Player 8.0.22.0,Neolane v6 6.1.0 Build 8172,8.0.22.0,-,28,30,100,46,43,0,0,cpe:/a:adobe:flash_player:8.0.22.0,27,27,0
4767,5130,adobe,acrobat_reader,Adobe Acrobat Reader 11.0.7,Adobe Acrobat 9.5.5 - CPSID_83708,11.0.7,-,50,52,100,52,50,0,0,cpe:/a:adobe:acrobat_reader:11.0.7,27,33,0


In [14]:
# Check that all rows are labelled

# (Should return "False")

df_label_software['match'].isnull().any()

False

In [15]:
# Format training data as "X" == "features, "y" == target.
# The target value is the 1st column.
df_match_train1 = df_label_software[['match', 'fz_ratio', 'fz_ptl_ratio', 'fz_tok_set_ratio',
        'fz_ptl_tok_sort_ratio', 'fz_uwratio', 'fz_rel_ratio',
        'fz_rel_ptl_ratio', 'titlX_len', 'DsplyNm0_len']]

# Convert into 2 numpy arrays for the scikit-learn ML classification algorithms.
np_match_train1 = np.asarray(df_match_train1)
X, y = np_match_train1[:, 1:], np_match_train1[:, 0]

print(X.shape, y.shape)

(22262, 9) (22262,)


## Use a grid search to tune the ML algorithm

As before, the classification algorithm needs to be tuned for optimal performance with the data.

This is done using a randomized grid search.
This code was modified from the [scikit-learn sample code](http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html#sphx-glr-auto-examples-model-selection-randomized-search-py).

In [16]:
#	Now find optimum parameters for model using Grid Search

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# build a classifier
clf = RandomForestClassifier()

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            

# specify parameters and distributions to sample from
param_dist = {"n_estimators": sp_randint(20, 100),
              "max_depth": [3, None],
              "max_features": sp_randint(1,7),
              "min_samples_split": sp_randint(2,7),
              "min_samples_leaf": sp_randint(1, 7),
              "bootstrap": [True, False],
              "class_weight": ['balanced', None],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 40
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 48.46 seconds for 40 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.976 (std: 0.007)
Parameters: {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 4, 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 21}

Model with rank: 2
Mean validation score: 0.976 (std: 0.008)
Parameters: {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 2, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 32}

Model with rank: 2
Mean validation score: 0.976 (std: 0.009)
Parameters: {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'max_features': 4, 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 48}




## Run the ML classifier with optimum parameters on the test data

Based on the above, and ignoring default values, the optimum set of parameters would be something like the following:

    'bootstrap':True, 'min_samples_leaf': 3, 'n_estimators': 35, 'min_samples_split': 4, 'criterion':'gini', 'max_features': 4, 'max_depth: None, 'class_weight': None

The RandomForest classifier is now trained on the test data to produce the model.

In [19]:
clf = RandomForestClassifier(
    bootstrap=True,
    min_samples_leaf=3,
    n_estimators=35,
    min_samples_split=4,
    criterion='gini',
    max_features=4,
    max_depth=None,
    class_weight=None
)

# Train model on original training data
clf.fit(X, y)

# save model for future use

from sklearn.externals import joblib
joblib.dump(clf, '/home/jovyan/work/vulnmine/vulnmine_data/software_classif_trained_Rdm_Forest.pkl.z') 

['/home/jovyan/work/vulnmine/vulnmine_data/software_classif_trained_Rdm_Forest.pkl.z']

In [20]:
# Test loading

clf = joblib.load('/home/jovyan/work/vulnmine/vulnmine_data/software_classif_trained_Rdm_Forest.pkl.z')